In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d atharv1610/terrain-recognition

terrain-recognition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset = '/content/terrain-recognition.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [5]:
!ls

'Data Main'   kaggle.json   sample_data   terrain-recognition.zip


In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split

In [7]:
grassy = os.listdir('/content/Data Main/train/Grassy')
print(grassy[0:5])
print(grassy[-5:])

['Grassy (7)_16_21.png', 'Grassy (12)_13_29.png', 'Grassy (11)_6_21.png', 'Grassy (5)_3_25.png', 'Grassy (5)_3_28.png']
['Grassy (14)_6_16.png', 'Grassy (11)_11_14.png', 'Grassy (11)_15_29.png', 'Grassy (19)_7_10.png', 'Grassy (18)_9_29.png']


In [8]:
print(len(grassy))

8512


In [9]:
marshy = os.listdir('/content/Data Main/train/Marshy')
rocky = os.listdir('/content/Data Main/train/Rocky')
sandy = os.listdir('/content/Data Main/train/Sandy')

In [10]:
print(len(marshy))

7727


In [11]:
grassy_lables=[0]*(len(grassy))
marshy_lables=[1]*len(marshy)
rocky_lables=[2]*len(rocky)
sandy_lables=[3]*len(sandy)

In [12]:
labels=grassy_lables+marshy_lables+rocky_lables+sandy_lables

In [13]:
print(len(labels))

31571


In [14]:
# convert images to numpy arrays+
grassy_path = '/content/Data Main/train/Grassy/'

data = []

for img_file in grassy:

  image = Image.open(grassy_path + img_file)
  image = image.resize((64,64))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)



marshy_path = '/content/Data Main/train/Marshy/'


for img_file in marshy:

  image = Image.open(marshy_path + img_file)
  image = image.resize((64,64))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)


rocky_path = '/content/Data Main/train/Rocky/'


for img_file in rocky:

  image = Image.open(rocky_path + img_file)
  image = image.resize((64,64))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)

sandy_path = '/content/Data Main/train/Sandy/'


for img_file in sandy:

  image = Image.open(sandy_path + img_file)
  image = image.resize((64,64))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)

In [15]:
X = np.array(data)
Y = np.array(labels)

In [16]:
print(X.shape)
print(Y.shape)

(31571, 64, 64, 3)
(31571,)


In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=2)

In [18]:
print(X_train[0])

[[[ 57  47  54]
  [ 58  49  53]
  [ 60  48  56]
  ...
  [135 116 117]
  [157 141 141]
  [137 120 123]]

 [[ 60  51  55]
  [ 72  63  65]
  [ 79  67  71]
  ...
  [ 87  71  77]
  [124 109 115]
  [130 116 123]]

 [[ 60  57  62]
  [ 71  68  73]
  [ 84  80  85]
  ...
  [104  89  94]
  [ 96  81  88]
  [ 81  69  74]]

 ...

 [[ 64  52  61]
  [ 68  55  65]
  [ 77  65  75]
  ...
  [ 62  54  59]
  [ 75  67  72]
  [ 85  75  80]]

 [[ 66  55  64]
  [ 68  56  66]
  [ 75  63  74]
  ...
  [ 71  62  68]
  [ 74  64  69]
  [ 85  74  79]]

 [[ 69  57  66]
  [ 71  60  70]
  [ 69  59  70]
  ...
  [ 81  70  77]
  [ 85  74  81]
  [ 82  74  78]]]


In [19]:
print(X.shape, X_train.shape, X_test.shape)

(31571, 64, 64, 3) (22099, 64, 64, 3) (9472, 64, 64, 3)


In [20]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [21]:
import tensorflow as tf
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow as tf
from tensorflow.keras.models import Sequential  # Import Sequential from keras.models
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout

In [22]:
num_of_classes = 4

model = keras.Sequential()

model.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(64,64,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))


model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))


model.add(keras.layers.Dense(num_of_classes, activation='softmax'))

In [23]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [24]:
# history = model.fit(train_data,
#                     batch_size=32,
#                     epochs=15,
#                     steps_per_epoch=len(train_data),
#                     validation_data=validation_data,
#                     validation_steps=len(validation_data))
history = model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs=15)

Epoch 1/15
622/622 [==============================] - 15s 8ms/step - loss: 0.7181 - acc: 0.7091 - val_loss: 0.4571 - val_acc: 0.8190
Epoch 2/15
622/622 [==============================] - 4s 7ms/step - loss: 0.4928 - acc: 0.8172 - val_loss: 0.4150 - val_acc: 0.8520
Epoch 3/15
622/622 [==============================] - 4s 6ms/step - loss: 0.4301 - acc: 0.8436 - val_loss: 0.3278 - val_acc: 0.8778
Epoch 4/15
622/622 [==============================] - 4s 6ms/step - loss: 0.3601 - acc: 0.8689 - val_loss: 0.2855 - val_acc: 0.8882
Epoch 5/15
622/622 [==============================] - 4s 7ms/step - loss: 0.3152 - acc: 0.8870 - val_loss: 0.3986 - val_acc: 0.8588
Epoch 6/15
622/622 [==============================] - 4s 6ms/step - loss: 0.2960 - acc: 0.8974 - val_loss: 0.2323 - val_acc: 0.9158
Epoch 7/15
622/622 [==============================] - 4s 7ms/step - loss: 0.2455 - acc: 0.9159 - val_loss: 0.3329 - val_acc: 0.8805
Epoch 8/15
622/622 [==============================] - 5s 9ms/step - loss: 0

In [25]:
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print('Test Accuracy =', accuracy)

296/296 [==============================] - 1s 4ms/step - loss: 0.2147 - acc: 0.9330
Test Accuracy = 0.9329603314399719
